# Introduction
This data set is include woman patients health features who is diabetes or not

Content:
1. [Load and Check Data](#1)
1. [Numeric Variable](#2)
1. [Outlier Detection](#3)
1. [Missing Value](#4)
    * [Find Missing Value](#5)
    * [Fill Missing Value](#6)
1. [Visualization](#7)
    * [Glucose Mean -- Label](#8)
    * [Insulin Mean -- Label](#9)
    * [BMI Mean -- Label](#10)
    * [BloodPressure Mean -- Label](#11)
    * [Pregnancies Mean -- Label](#12)
    * [SkinThickness Mean -- Label](#13)
    * [Age Mean -- Label](#14)
    * [DiabetesPedigreeFunction Mean -- Label](#15)
1. [Modelling](#16)
    * [Train - Test Split](#17)
    * [Simple Logistid Regression](#18)
    * [Hyperparameter Tuning -- Griid Search -- Cross Validation](#19)
    * [Ensemble Modeling](#20)
    * [Predict Test Data](#21)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")
import seaborn as sns

from collections import Counter

import warnings
warnings.filterwarnings("ignore")


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="1"></a><br>
## Load and Check Data

In [ ]:
data = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
data.columns

* Outcome feature is represent patients diabetes results
* Lets change this to Label

In [ ]:
data.head()

In [ ]:
data["Label"] = data.Outcome.values
data.drop(labels=["Outcome"], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
len(data)

There is 768 patients

In [ ]:
data.describe()

<a id="2"></a><br>
## Variable Describe

1. Pregnancies: Number of times pregnant
1. Glucose: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
1. BloodPressure: Diastolic Blood Pressure (mm Hg)
1. SkinThickness: Triceps skin fold thickness (mm)
1. Insulin: 2-Hour serum insulin (mu U/ml)
1. BMI: Body mass index (weight in kg/(height in m)^2)
1. DiabetesPedigreeFunction: Diabetes pedigree function
1. Age: Age (years)
1. Outcome: Class variable (0 or 1) 268 of 768 are 1, the others are 0

In [ ]:
data.info()

* float64 (2): BMI, DıabetesPedigreeFunction
* int64 (7): Pregnancies, Glucosei BloodPressure, SkinThickness, Insulin, Age, Outcome

<a id="2"></a><br>
# Numerical Variable
* This data set has not include categorical variable so lets look short the numerical variables

In [ ]:
def plot_hist(variable):
    plt.figure(figsize=(9,3))
    plt.hist(data[variable], bins=40)
    plt.xlabel(variable)
    plt.ylabel("Frequancies")
    plt.title(f"{variable} distrubition with hist")

In [ ]:
numeric_variables = ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"]
for variable in numeric_variables:
    plot_hist(variable)

<a id="3"></a><br>
# Outlier Detection

In [ ]:
def detect_outliers(df, features):
    outlier_indices = []
    
    for c in features:
        # 1st quartile
        Q1 = np.percentile(df[c], 25)
        # 3rd quartile
        Q3 = np.percentile(df[c], 75)
        # IQR
        IQR = Q3 - Q1
        # Outlier Step
        outlier_step = IQR * 1.5
        # detect outlier and their indices
        outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)]. index
        # store indices
        outlier_indices.extend(outlier_list_col)
    
    # print() outlier_indices
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list((i for i, v in outlier_indices.items() if v > 2 ))
    
    return multiple_outliers

In [ ]:
data.iloc[detect_outliers(data, ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"])]

* There is no double outlier varieble in database
* We don't drop any varieble

In [ ]:
# drop the outlier variables
#data.drop(detect_outliers(data, ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"], axis=0)).reset_index(drop=True)

<a id="4"></a><br>
# Missing Value
* Find Missing Value
* Fill Missing Value

<a id="5"></a><br>
## Find Missing Value

* Missing value is 0 on this Database

In [ ]:
data.columns[data.isnull().any()]

In [ ]:
data.head()

In [ ]:
data[data==0].count()

<a id="6"></a><br>
## Fill Missing Value

* Pregnancies variable include 0 value but its not missing value this mean she have never been pregnant so we don't change this
* Glucose, BloodPressure, SkinThickness, Insulin, BMI has include missing values. Lets get fill this values

In [ ]:
data[data["Glucose"] == 0]

In [ ]:
plt.scatter(x=data["Age"], y=data["Glucose"])
plt.xlabel("Age")
plt.ylabel("Glucose")
plt.title("Age - Glucose Distribution")
plt.show()

In [ ]:
plt.scatter(x=data["BloodPressure"], y=data["Glucose"])
plt.xlabel("BloodPressure")
plt.ylabel("Glucose")
plt.title("BloodPressure - Glucose Distribution")
plt.show()

In [ ]:
mean = int(np.mean(data[data["Glucose"]!=0]["Glucose"]))
print("Mean :",mean)
data["Glucose"] = [i if i != 0 else mean for i in data["Glucose"]]

In [ ]:
data[data["Glucose"] == 0]

In [ ]:
# Fill BloodPressure
data[data["BloodPressure"] == 0]["BloodPressure"].count()

In [ ]:
mean = int(np.mean(data[data["BloodPressure"] != 0]["BloodPressure"]))
print("Mean :",mean)
data["BloodPressure"] = [i if i != 0 else mean for i in data["BloodPressure"]]

In [ ]:
data[data["BloodPressure"] == 0]["BloodPressure"].count()

In [ ]:
# Fill SkinThickness
data[data["SkinThickness"] == 0]["SkinThickness"].count()

In [ ]:
mean = int(np.mean(data[data["SkinThickness"] != 0]["SkinThickness"]))
print("Mean :",mean)
data["SkinThickness"] = [i if i != 0 else mean for i in data["SkinThickness"]]

In [ ]:
data[data["SkinThickness"] == 0]["SkinThickness"].count()

In [ ]:
# Fill Insulin
data[data["Insulin"] == 0]["Insulin"].count()

In [ ]:
mean = int(np.mean(data[data["Insulin"] != 0]["Insulin"]))
print("Mean :",mean)
data["Insulin"] = [i if i != 0 else mean for i in data["Insulin"]]

In [ ]:
data[data["Insulin"] == 0]["Insulin"].count()

In [ ]:
# Fill BMI
data[data["BMI"] == 0]["BMI"].count()

In [ ]:
mean = int(np.mean(data[data["BMI"] != 0]["BMI"]))
print("Mean :",mean)
data["BMI"] = [i if i != 0 else mean for i in data["BMI"]]

In [ ]:
data[data["BMI"] == 0]["BMI"].count()

<a id="7"></a><br>
# Visualization

In [ ]:
list1 = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "Label"]
sns.heatmap(data[list1].corr(), annot = True, fmt=".2f")
plt.show()

Glucose and BMI features seems to have corelation with Label feature (0.49, 0.31)

<a id="8"></a><br>
## Glucose Mean -- Label

In [ ]:
# 0 = Non Diabetes , 1 = Diabetes
zero_mean = data[data["Label"] == 0]["Glucose"].mean()
one_mean = data[data["Label"] == 1]["Glucose"].mean()
zero_count = data[data["Label"]==0]["Label"].count()
one_count = data[data["Label"]==1]["Label"].count()
g = sns.barplot(x=data["Label"].value_counts().index, y=[zero_mean,one_mean], label=("0 count:",zero_count,"1 Count:",one_count))
g.set_ylabel("Glucose Means")
g.legend()
plt.tight_layout()
plt.show()

* Diabetes Patient's glucose level more than not Diabetes Patient's

<a id="9"></a><br>
## Insulin Mean -- Label

In [ ]:
# 0 = Non Diabetes , 1 = Diabetes
zero_mean = data[data["Label"] == 0]["Insulin"].mean()
one_mean = data[data["Label"] == 1]["Insulin"].mean()
zero_count = data[data["Label"]==0]["Label"].count()
one_count = data[data["Label"]==1]["Label"].count()
g = sns.barplot(x=data["Label"].value_counts().index, y=[zero_mean,one_mean], label=("0 count:",zero_count,"1 Count:",one_count))
g.set_ylabel("Insulin Means")
g.legend()
plt.tight_layout()
plt.show()

Diabetes Patient's Insulin Levels more than Non Diabetes Patient's

<a id="10"></a><br>
## BMI Mean -- Label

In [ ]:
# 0 = Non Diabetes , 1 = Diabetes
zero_mean = data[data["Label"] == 0]["BMI"].mean()
one_mean = data[data["Label"] == 1]["BMI"].mean()
zero_count = data[data["Label"]==0]["Label"].count()
one_count = data[data["Label"]==1]["Label"].count()
g = sns.barplot(x=data["Label"].value_counts().index, y=[zero_mean,one_mean], label=("0 count:",zero_count,"1 Count:",one_count))
g.set_ylabel("BMI Means")
g.legend()
plt.tight_layout()
plt.show()

Diabetes Patient's BMI Values more than Non Diabetes Patient's

<a id="11"></a><br>
## BloodPressure Mean -- Label

In [ ]:
# 0 = Non Diabetes , 1 = Diabetes
zero_mean = data[data["Label"] == 0]["BloodPressure"].mean()
one_mean = data[data["Label"] == 1]["BloodPressure"].mean()
zero_count = data[data["Label"]==0]["Label"].count()
one_count = data[data["Label"]==1]["Label"].count()
g = sns.barplot(x=data["Label"].value_counts().index, y=[zero_mean,one_mean], label=("0 count:",zero_count,"1 Count:",one_count))
g.set_ylabel("BloodPressure Means")
g.legend()
plt.tight_layout()
plt.show()

Diabetes Patient's Insulin Levels little more than Non Diabetes Patient's

<a id="12"></a><br>
## Pregnancies Mean -- Label

In [ ]:
# 0 = Non Diabetes , 1 = Diabetes
zero_mean = data[data["Label"] == 0]["Pregnancies"].mean()
one_mean = data[data["Label"] == 1]["Pregnancies"].mean()
zero_count = data[data["Label"]==0]["Label"].count()
one_count = data[data["Label"]==1]["Label"].count()
g = sns.barplot(x=data["Label"].value_counts().index, y=[zero_mean,one_mean], label=("0 count:",zero_count,"1 Count:",one_count))
g.set_ylabel("Pregnancies Means")
g.legend()
plt.tight_layout()
plt.show()

Lower pregnancy average is good about Diabetes

<a id="13"></a><br>
## SkinThickness Mean -- Label

In [ ]:
# 0 = Non Diabetes , 1 = Diabetes
zero_mean = data[data["Label"] == 0]["SkinThickness"].mean()
one_mean = data[data["Label"] == 1]["SkinThickness"].mean()
zero_count = data[data["Label"]==0]["Label"].count()
one_count = data[data["Label"]==1]["Label"].count()
g = sns.barplot(x=data["Label"].value_counts().index, y=[zero_mean,one_mean], label=("0 count:",zero_count,"1 Count:",one_count))
g.set_ylabel("SkinThickness Means")
g.legend()
plt.tight_layout()
plt.show()

<a id="14"></a><br>
## Age Mean -- Label

In [ ]:
# 0 = Non Diabetes , 1 = Diabetes
zero_mean = data[data["Label"] == 0]["Age"].mean()
one_mean = data[data["Label"] == 1]["Age"].mean()
zero_count = data[data["Label"]==0]["Label"].count()
one_count = data[data["Label"]==1]["Label"].count()
g = sns.barplot(x=data["Label"].value_counts().index, y=[zero_mean,one_mean], label=("0 count:",zero_count,"1 Count:",one_count))
g.set_ylabel("Age Means")
g.legend()
plt.tight_layout()
plt.show()

Old people are under risk for Diabetes more than younger people

<a id="15"></a><br>
## DiabetesPedigreeFunction Mean -- Label

In [ ]:
# 0 = Non Diabetes , 1 = Diabetes
zero_mean = data[data["Label"] == 0]["DiabetesPedigreeFunction"].mean()
one_mean = data[data["Label"] == 1]["DiabetesPedigreeFunction"].mean()
zero_count = data[data["Label"]==0]["Label"].count()
one_count = data[data["Label"]==1]["Label"].count()
g = sns.barplot(x=data["Label"].value_counts().index, y=[zero_mean,one_mean], label=("0 count:",zero_count,"1 Count:",one_count))
g.set_ylabel("DiabetesPedigreeFunction Means")
g.legend()
plt.tight_layout()
plt.show()

<a id="16"></a><br>
# Modelling

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

<a id="17"></a><br>
## Train - Test Split

In [ ]:
# We will use this test data after select the model, parameters etc.

test = data.iloc[700:]
test_label= test["Label"]
test.drop(labels=["Label"], axis=1, inplace=True)

In [ ]:
train_df = data.iloc[:700]
X_train = train_df.drop(labels=["Label"], axis=1)
Y_train = train_df["Label"]

x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
print(f"x_train size: {len(x_train)}")
print(f"y_train size: {len(y_train)}")
print(f"x_test size: {len(x_test)}")
print(f"y_test size: {len(y_test)}")

<a id="18"></a><br>
## Simple Logistid Regression

In [ ]:
lr = LogisticRegression()
lr.fit(x_train,y_train)
acc_lr_train = round(lr.score(x_train, y_train)*100, 2)
acc_lr_test = round(lr.score(x_test, y_test)*100, 2)
print("Logistic Regression Train Accuracy: %", acc_lr_train)
print("Logistic Regression Test Accuracy: %", acc_lr_test)

<a id="19"></a><br>
## Hyperparameter Tuning -- Griid Search -- Cross Validation

We will compare 5 ml classifier and evaluate mean accuacy of each of them by stratified cros validation.

* Decision Tree
* SVM
* Random Forest
* KNN
* Logistic Regression

In [ ]:
random_state=42
classifier=[DecisionTreeClassifier(random_state=random_state),
           SVC(random_state=random_state),
           RandomForestClassifier(random_state=random_state),
           LogisticRegression(random_state=random_state),
           KNeighborsClassifier()]

dt_param_grid = {"min_samples_split":range(10,500,20),
           "max_depth":range(1,20,2)}

svc_param_grid = {"kernel":["rbf"],
            "gamma":[0.001, 0.01, 0.1, 1],
            "C":[1,10,50,100,200,300,1000]}

rf_param_grid = {"max_features":[1,3,10],
                "min_samples_split":[2,3,10],
                "min_samples_leaf":[1,3,10],
                "bootstrap":[False],
                "n_estimators":[100,300],
                "criterion":["gini"]}

logreg_param_grid = {"C":np.logspace(-3,3,7),
                    "penalty":["l1","l2"]}

knn_param_grid = {"n_neighbors":np.linspace(1,19,10, dtype=int).tolist(),
                 "weights":["uniform","distance"],
                 "metric":["euclidean","manhattan"]}

classifier_params = [dt_param_grid, 
                    svc_param_grid, 
                    rf_param_grid, 
                    logreg_param_grid, 
                    knn_param_grid]

In [ ]:
cv_result = []
best_estimators = []
for i in range(len(classifier)):
    clf = GridSearchCV(classifier[i], param_grid = classifier_params[i], cv=StratifiedKFold(n_splits=10), scoring="accuracy", n_jobs=-1, verbose=1)
    clf.fit(x_train,y_train)
    cv_result.append(clf.best_score_)
    best_estimators.append(clf.best_estimator_)
    print(cv_result[i])

In [ ]:
print(cv_result)
cv_results = pd.DataFrame({"Cross Validaiton Means":cv_result, "Models":["DecisionTree","SVM","RandomForest","LogisticRegression","KNN"]})
g = sns.barplot(x="Cross Validaiton Means", y = "Models", data = cv_results)
g.set_xlabel("Model Accuracy")
g.set_title("Cross Validation Score")
plt.show()

<a id="20"></a><br>
## Ensemble Modeling

In [ ]:
votingC = VotingClassifier(estimators = [("dt",best_estimators[0]),
                                        ("rf",best_estimators[2]),
                                        ("logreg",best_estimators[3])],
                                        voting="soft",
                                        n_jobs=-1)
votingC = votingC.fit(x_train,y_train)
print(accuracy_score(votingC.predict(x_test),y_test))

<a id="21"></a><br>
## Predict Test Data

In [ ]:
prediction = votingC.predict(test)
print(accuracy_score(prediction, test_label))